In [ ]:
from typing import *
import ipywidgets as wi
import pickle
from random import Random
from script_util import root, fname2id

In [ ]:
PX = 128

In [ ]:
def load_wimage(
    fname: str,
    PX: int = PX,
    ):
    try:
        with open(fname, "rb") as f:
            buf = f.read()
        image = wi.Image(value=buf, format='png', width=PX, height=PX)
    except Exception as e:
        image = wi.Image(width=PX, height=PX)
    return image

In [ ]:
class Pannel(wi.VBox):
    def on_click(self, event) -> None:
        if not event["new"]:
            self.bag.add(self.fname)
        else:
            self.bag.discard(self.fname)
        self.output.clear_output()
        with self.output:
            print(self.id, len(self.bag))
    def __init__(
        self,
        fname: str,
        bag: Set[str],
        output: wi.Output,
        PX: int = PX,
    ) -> None:
        self.fname = fname
        self.bag = bag
        self.output = output
        self.PX = PX
        self.id = fname2id(fname)
        self.image = load_wimage(fname, self.PX)
        self.button = wi.ToggleButton(description=self.id, layout=wi.Layout(width=f"{PX}px"), value=self.fname not in bag, button_style="info")
        self.button.observe(self.on_click, "value")
        super().__init__([self.image, self.button])

In [ ]:
class Viewer(wi.Box):
    def __len__(self) -> int:
        return (len(self.fnames)-1)//self.N+1
    def __getitem__(self, i: int) -> List[Pannel]:
        assert i in range(len(self))
        return [
            Pannel(fname, self.bag, self.output, self.PX)
            for fname in self.fnames[i*self.N:(i+1)*self.N]
        ]
    def refresh(self) -> None:
        self.status.clear_output()
        with self.status:
            print(f"{self.i}/{len(self)}", len(self.fnames))
        self.children = tuple(self[self.i]+[self.controller])
    def on_click_p(self, event) -> None:
        self.i -= 1; self.i %= len(self);
        self.select.value = self.i
    def on_click_n(self, event) -> None:
        self.i += 1; self.i %= len(self);
        self.select.value = self.i
    def on_select(self, event) -> None:
        self.i = event["new"]; self.refresh()
    def is_saved(self) -> bool:
        return  (root/"viewer"/f"{self.name}.pkl").is_file()
    def save(self, event=None) -> None:
        self.status.clear_output()
        with self.status:
            print("saving...")
        (root/"viewer").mkdir(exist_ok=True)
        with open(root/"viewer"/f"{self.name}.pkl", "wb") as f:
            pickle.dump(self.bag, f, protocol=4)
        self.status.clear_output()
        with self.status:
            print(f"{self.i}/{len(self)}", len(self.fnames), "saved")
    def load(self, event=None) -> None:
        if not self.is_saved():
            self.status.clear_output()
            with self.status:
                print(f"{self.i}/{len(self)}", len(self.fnames), "not saved")
            return
        self.status.clear_output()
        with self.status:
            print("loading...")
        with open(root/"viewer"/f"{self.name}.pkl", "rb") as f:
            self.bag = pickle.load(f)
        self.refresh()
        self.status.clear_output()
        with self.status:
            print(f"{self.i}/{len(self)}", len(self.fnames), "loaded")
    def __init__(
        self,
        fnames: Iterable[str],
        N: int = 23,
        seed: int = 0,
        name: str = '',
        key = None,
        PX: int = PX,
    ) -> None:
        self.N = N
        self.seed = seed
        self.PX = PX
        self.fnames = list(map(str, fnames))
        if len(self.fnames) == 0:
            super().__init__()
            print("0");  return
        if key == None:
            self.fnames.sort()
            Random(self.seed).shuffle(self.fnames)
        elif key == "nosort":
            pass
        else:
            self.fnames.sort(key=lambda fname: key(fname2id(fname)))
        self.fnames = tuple(self.fnames)
        if name == '':
            self.name = hash(self.fnames)
        else:
            self.name = name
        self.i = 0
        self.bag = set()
        self.select = wi.Select(options=range(len(self)), layout=wi.Layout(width=f"{PX//2}px", height=f"{round(PX)*0.95}px"))
        self.button_n = wi.Button(description="next", layout=wi.Layout(width=f"{PX//2}px"))
        self.button_p = wi.Button(description="prev", layout=wi.Layout(width=f"{PX//2}px"))
        self.button_s = wi.Button(description="save", layout=wi.Layout(width=f"{PX//2}px"))
        self.button_l = wi.Button(description="load", layout=wi.Layout(width=f"{PX//2}px"))
        self.status = wi.Output(layout=wi.Layout(width=f"{PX}px"))
        self.output = wi.Output(layout=wi.Layout(width=f"{PX}px"))
        self.controller = wi.VBox([
            wi.HBox([wi.VBox([
                self.button_n,
                self.button_p,
                self.button_s,
                self.button_l,
            ]), self.select]),
            self.status,
            self.output,
        ])
        self.layout = layout = wi.Layout(flex_flow="row wrap")
        self.button_n.on_click(self.on_click_n)
        self.button_p.on_click(self.on_click_p)
        self.button_s.on_click(self.save)
        self.button_l.on_click(self.load)
        self.select.observe(self.on_select, "value")
        super().__init__()
        if self.is_saved():
            self.load()
        else:
            self.refresh()

In [ ]:
class Manager(Viewer):
    MANAGED = dict()
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        Manager.MANAGED[kwargs["name"]] = self
    @classmethod
    def allbag(cls):
        retval = set()
        for k, v in cls.MANAGED.items():
            print(k, len(v.bag))
            retval |= set(map(fname2id, v.bag))
        return retval